### Import Modules & Data

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [14]:
churn_df = pd.read_csv('churn_data.csv')
#churn_df.info()
churn_df.head()

,Customer ID,Gender,Senior Citizen,Partner,Dependents,tenure,Phone Service,Multiple Lines,Internet Service,Online Security,...,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn
0,7590-VHVEA,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
2,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
3,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
4,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No


As the data has a lot of object types :

Dropping 'Churn' & 'Customer ID' for X & using 

pandas.get_dummies() -
    Converts categorical variable into dummy/indicator variables
    Return dummy-coded data

Get 'Churn' column for Y -
    Convert 'Yes' or 'No' -> 1 or 0

In [10]:
x = pd.get_dummies(churn_df.drop(['Churn', 'Customer ID'], axis=1))
y = churn_df['Churn'].apply(lambda x : 1 if x=='Yes' else 0)

y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 7044 entries, 0 to 7043
Series name: Churn
Non-Null Count  Dtype
--------------  -----
7044 non-null   int64
dtypes: int64(1)
memory usage: 55.2 KB


### Train Test Split

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)


In [17]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5635 entries, 6047 to 2682
Columns: 6575 entries, Senior Citizen to Total Charges_999.9
dtypes: float64(1), int64(2), uint8(6572)
memory usage: 35.5 MB


In [18]:
y_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 5635 entries, 6047 to 2682
Series name: Churn
Non-Null Count  Dtype
--------------  -----
5635 non-null   int64
dtypes: int64(1)
memory usage: 88.0 KB


In [19]:
print('x_train shape : ', x_train.shape)
print('y_train shape : ', y_train.shape)
print('x_test shape : ', x_test.shape)
print('y_test shape : ', y_test.shape)

x_train shape :  (5635, 6575)
y_train shape :  (5635,)
x_test shape :  (1409, 6575)
y_test shape :  (1409,)


### Import DL Dependencies

In [20]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

Build Model & Compile it

In [25]:
model = Sequential([
    Dense(units=32, activation='relu', input_dim=len(x_train.columns)),
    Dense(units=64, activation='relu'),
    Dense(units=1, activation='sigmoid')
])


In [26]:
model.compile(loss='binary_crossentropy', optimizer='sgd',metrics='accuracy')


### Fit, Predict & Evaluate

In [27]:
model.fit(x_train, y_train, epochs=400, batch_size=32)

Epoch 1/400
177/177 [==============================] - 1s 3ms/step - loss: 0.5093 - accuracy: 0.7512
Epoch 2/400
177/177 [==============================] - 1s 3ms/step - loss: 0.4892 - accuracy: 0.7714
Epoch 3/400
177/177 [==============================] - 1s 3ms/step - loss: 0.4817 - accuracy: 0.7775
Epoch 4/400
177/177 [==============================] - 1s 3ms/step - loss: 0.4750 - accuracy: 0.7824
Epoch 5/400
177/177 [==============================] - 1s 3ms/step - loss: 0.4714 - accuracy: 0.7826
Epoch 6/400
177/177 [==============================] - 1s 4ms/step - loss: 0.4676 - accuracy: 0.7819
Epoch 7/400
177/177 [==============================] - 1s 3ms/step - loss: 0.4663 - accuracy: 0.7846
Epoch 8/400
177/177 [==============================] - 1s 3ms/step - loss: 0.4644 - accuracy: 0.7844
Epoch 9/400
177/177 [==============================] - 1s 3ms/step - loss: 0.4617 - accuracy: 0.7828
Epoch 10/400
177/177 [==============================] - 1s 3ms/step - loss: 0.4603 - accura

##### Predict Outcomes for x_test :

TensorFlow will output continuous value from 0 to 1. So, those values has to be converted to 0s and 1s.

In [32]:
y_hat = model.predict(x_test)
y_hat = [0 if val < 0.5 else 1 for val in y_hat]

45/45 [==============================] - 0s 2ms/step


##### Accuracy Score

Score for y_test (test output) & y_hat (predicted output)

**Note :** Accuracy score of 0.8 or 80% is commonly considered good!

In [33]:
accuracy_score(y_test, y_hat)

0.7828246983676366

### Saving & Reloading

In [34]:
# This will save the model in the folder name specified
model.save('tf_churn_model')

INFO:tensorflow:Assets written to: tf_churn_model\assets


In [35]:
# This will delete the model from the memory, since we are going to reload the saved model
del model

Load Model

In [37]:
model = load_model('tf_churn_model')